In [0]:
# Set the name of the spark job
nameOfSparkJob=os.getenv('JUPYTERHUB_USER') + "-pm-counters-validation-2"
#print(os.getenv('JUPYTERHUB_USER'))
print(nameOfSparkJob+ " started")

# Start the SparkSession
spark = (
    SparkSession
    .builder
    .appName(nameOfSparkJob)
    .enableHiveSupport()
    .config("spark.dynamicAllocation.minExecutors", 12)  # 2 is generally recommended
    .config("spark.dynamicAllocation.maxExecutors", 30)  # 2 is generally recommended
    .config("spark.executor.memory", "8g")  # default is 3 - adjust if needed
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

parquetFiles = r"/home/jovyan/mapr-prod/rdi/luigi/pm/parsed/customerid=200393/date=2025-01-15*"
df = spark.read.parquet(parquetFiles)
#print(df.printSchema())

from pyspark.sql import functions as F

# Filter for moClass "NRCellDU" and specific counters
result = (
    df.filter(F.col("moClass") == "")  # Filter for moClass as "NRCellDU"
    .select("moClass", F.explode("Counters").alias("counterKey", "counterValue"))  # Explode Counters
    .filter(
        F.col("counterKey").rlike(
            "pmEbsnDrbExclThpDlRelativeLowVol|pmEbsnDrbExclThpDlShortTimeLowVol|pmEbsnMacDrbThpDlMbbLowVolDistr|pmEbsnMacUeThpUlMbbLowVolDistr|pmEbsnUeExclThpUlRelativeLowVol|pmEbsnUeExclThpUlShortTimeLowVol"
        )
    )
    .select("counterKey", "counterValue")  # Select the required columns
    .distinct()  # Ensure unique combinations
)

# Show the result
result.show(100, False)